In [1]:
import json
import os
import pickle
from pprint import pprint

from sklearn.model_selection import train_test_split
#train_test_split(X, y, test_size=0.2, random_state=rand_seed)


all_tokens = {}

prep_path = 'data/preprocessed/'
graphs_dir = 'data/graphs/'
prep_dirs = ['CuckooClean/', 'CuckooCleanHippo/', 'CuckooCleanPippo/', 'CuckooVirusShare/']
files_a, files_b = 0, 300
labels = [False, False, False, True]
for prep_dir in prep_dirs:
    if prep_dir != 'CuckooVirusShare/':
        continue
    print('Initializing for ' + prep_dir)
    all_calls = {}
    processed_files_num = 0
    file_num = 0
    num_of_traces = 0
    not_one_traces = []
    for filename in sorted([x for x in os.listdir(prep_path + prep_dir) if x.endswith('.json')])[files_a: files_b]:
        with open(prep_path + prep_dir + filename) as json_file:
            proc_traces = json.load(json_file)
        if len(proc_traces) != 1:
            not_one_traces.append(prep_path + prep_dir + filename)
        num_of_traces += len(proc_traces)
        for calls_list in proc_traces:
            g_calls = {}
            g_tokens = {}
            for log in calls_list["calls"]:
                if len(log) != 1:
                    print('Log record {} len not 1 in {}'.format(str(log), prep_path + prep_dir + filename))
                    continue

                call_name = next(iter(log))
                call_args = log[call_name]

                if not call_name in g_calls:
                    g_calls[call_name] = set()

                for call_arg in call_args:
                    str_arg = str(call_arg)
                    g_calls[call_name].add(str_arg)
                    if not str_arg in g_tokens:
                        g_tokens[str_arg] = set()
                    g_tokens[str_arg].add(call_name)
                    if not str_arg in all_tokens:
                        all_tokens[str_arg] = 1
                    else:
                        all_tokens[str_arg] += 1

                if not call_name in all_calls:
                    all_calls[call_name] = 1
                else:
                    all_calls[call_name] += 1
                    
            if not os.path.exists(graphs_dir + prep_dir):
                os.makedirs(graphs_dir + prep_dir)
            
            graph = {'g_calls':g_calls, 'g_tokens': g_tokens}
            with open(graphs_dir + prep_dir + filename[0] + '_' + str(file_num) + '.json', 'wb') as out_file:
                pickle.dump(graph, out_file, pickle.HIGHEST_PROTOCOL)
            file_num += 1
            if file_num % 100 == 0:
                print('Processed {} traces, {} files'.format(file_num, processed_files_num))
                
            
        processed_files_num += 1
        #if processed_files_num > 200:
        #    print('200 files gone')
        #    break
    
    
    with open(graphs_dir + prep_dir + 'all_calls_a' + str(files_a) + '_b' + str(files_b) + '.json', 'wb') as out_file:
        pickle.dump(all_calls, out_file, pickle.HIGHEST_PROTOCOL)     
    print('All calls written')
    break
    with open(graphs_dir + prep_dir + 'all_tokens_a' + str(files_a) + '_b' + str(files_b) + '.json', 'wb') as out_file:
        pickle.dump(all_tokens, out_file, pickle.HIGHEST_PROTOCOL)        
    print('All tokens written')
    
#hippo[0:100] last file_num was 566
#pippo[0:0] last file_num was 0
#virus[0:300] last file_num was 588
         


Initializing for CuckooVirusShare/
Processed 100 traces, 46 files
Processed 200 traces, 100 files
Processed 300 traces, 159 files
Processed 400 traces, 202 files
Processed 500 traces, 252 files
All calls written


In [3]:
print(file_num)
print(processed_files_num)


NameError: name 'file_num' is not defined

In [4]:
print(type(all_tokens))
print(len(all_tokens))
print(sorted(all_tokens.values())[-1])


<class 'dict'>
307721
2233437


In [7]:
top_tokens = { key:value for (key,value) in all_tokens.items() if value > 5}
print(type(top_tokens))
print(len(top_tokens))
#print(list(top_tokens.items()))
print(list(top_tokens.keys())[0:5])

<class 'dict'>
33280
['0x00000000', '0x80000000', 'HKEY_LOCAL_MACHINE\\Software\\Microsoft\\Windows NT\\CurrentVersion\\Image File Execution Options\\ntdll.dll', 'HKEY_LOCAL_MACHINE\\Software\\Microsoft\\Windows NT\\CurrentVersion\\Image File Execution Options\\KERNEL32.dll', '0x00000024']


In [8]:
with open(graphs_dir + 'CuckooVirusShare/' + 'top_tokens_a' + str(files_a) + '_b' + str(files_b) + '.json', 'wb') as out_file:
    pickle.dump(top_tokens, out_file, pickle.HIGHEST_PROTOCOL)   

In [1]:
from collections import Counter
import pickle

with open('data/graphs/' + 'CuckooClean/' + 'top_tokens' + '.json', 'rb') as out_file:
    top_tokens_clean = pickle.load(out_file)  
sum_top_tokens_clean = sum(top_tokens_clean.values())

with open('data/graphs/' + 'CuckooCleanHippo/' + 'top_tokens_a0_b100' + '.json', 'rb') as out_file:
    top_tokens_hippo = pickle.load(out_file)  
sum_top_tokens_hippo =sum(top_tokens_hippo.values())

with open('data/graphs/' + 'CuckooVirusShare/' + 'top_tokens_a0_b300' + '.json', 'rb') as out_file:
    top_tokens_virus = pickle.load(out_file) 
sum_top_tokens_virus = sum(top_tokens_virus.values())

print(sum_top_tokens_virus + sum_top_tokens_hippo + sum_top_tokens_clean)

54903805


In [2]:
top_tokens_all = dict(Counter(top_tokens_clean)+Counter(top_tokens_hippo)+Counter(top_tokens_virus))
print(sum(top_tokens_all.values()))

54903805


In [3]:
import operator
sorted_tokens = sorted(top_tokens_all.items(), key=operator.itemgetter(1), reverse=True)
print(len(sorted_tokens))
print(len(top_tokens_clean))
print(len(top_tokens_hippo))
print(len(top_tokens_virus))


204225
140505
120831
33280


In [4]:
print(sorted_tokens[:20])
print(sorted_tokens[14771])
print(sorted_tokens[14772])
print(sorted_tokens[65536])

final_tokens = [x[0] for x in sorted_tokens[:4096]]
print(len(final_tokens))
print(final_tokens[:20])

[('0', 8089857), ('0xffffffff', 1803864), ('1', 1706338), ('4096', 1704214), ('', 1251853), ('4', 1246827), ('0x00000000', 1112177), ('0x02000000', 583609), ('0x7e410000', 467096), ('USER32', 455931), ('2', 448764), ('0x00000001', 402251), ('#0', 356871), ('32', 353299), ('0x80000002', 344832), ('31', 331871), ('5', 304406), ('32768', 288379), ('0x80000000', 283490), ('1024', 279330)]
('0x3d153000', 100)
('appHelp.dll', 99)
('0x258b4000', 26)
4096
['0', '0xffffffff', '1', '4096', '', '4', '0x00000000', '0x02000000', '0x7e410000', 'USER32', '2', '0x00000001', '#0', '32', '0x80000002', '31', '5', '32768', '0x80000000', '1024']


In [5]:
tokens_map = {name : i for i, name in enumerate(final_tokens)}

In [7]:
import json
with open('data/graphs/' + 'tokens_map_4096.json', 'w') as outfile:
    json.dump(tokens_map, outfile)

In [32]:
with open('data/graphs/' + 'top_tokens_all.json', 'wb') as out_file:
    pickle.dump(final_tokens, out_file, pickle.HIGHEST_PROTOCOL)

In [33]:
with open('data/graphs/' + 'CuckooClean/' + 'all_calls' + '.json', 'rb') as out_file:
    all_calls_clean = pickle.load(out_file)  
sum_all_calls_clean = sum(all_calls_clean.values())

with open('data/graphs/' + 'CuckooCleanHippo/' + 'all_calls_a0_b100' + '.json', 'rb') as out_file:
    all_calls_hippo = pickle.load(out_file)  
sum_all_calls_hippo =sum(all_calls_hippo.values())

with open('data/graphs/' + 'CuckooVirusShare/' + 'all_calls_a0_b300' + '.json', 'rb') as out_file:
    all_calls_virus = pickle.load(out_file) 
sum_all_calls_virus = sum(all_calls_virus.values())
    
print(sum_all_calls_clean + sum_all_calls_hippo + sum_all_calls_virus)

all_calls_all = dict(Counter(all_calls_clean)+Counter(all_calls_hippo)+Counter(all_calls_virus))
print(sum(all_calls_all.values()))

21076930
21076930


In [35]:
print(len(all_calls_all))
sorted_calls = sorted(all_calls_all.items(), key=operator.itemgetter(1), reverse=True)
print(len(sorted_calls))
print(len(all_calls_clean))
print(len(all_calls_hippo))
print(len(all_calls_virus))

277
277
260
243
262


In [39]:
print(sorted_calls[:5])
final_calls = [x[0] for x in sorted_calls]
print(len(final_calls))
print(final_calls[-10:])

[('NtWriteFile', 2171124), ('GetSystemTimeAsFileTime', 1923418), ('NtReadFile', 1375270), ('GetAsyncKeyState', 1247014), ('NtClose', 1035732)]
277
['CryptEncrypt', 'NtDeleteKey', 'CryptProtectData', 'NotifyBootConfigStatus', 'CertOpenSystemStoreA', 'CreateDirectoryExW', 'RtlRemoveVectoredExceptionHandler', 'DeleteService', 'RtlDecompressBuffer', 'RtlCreateUserThread']


In [40]:
with open('data/graphs/' + 'calls_all.json', 'wb') as out_file:
    pickle.dump(final_calls, out_file, pickle.HIGHEST_PROTOCOL)